In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from scipy.signal import correlate2d
from sklearn.metrics import confusion_matrix

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle

In [3]:
from bayesee.generation import *

In [4]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [5]:
repo_path = Path.cwd().parents[0]
print(repo_path)

c:\Users\aqfra\Documents\GitRepo\bayesee


In [6]:
subject = "AZ_t1"
file_name = repo_path / f"data/covert-search/large-field/derived/est_dp_{subject}.npy"
subject_local_dp = np.load(file_name)

file_name = repo_path / f"data/covert-search/large-field/p3_data_{subject}.mat"
data = loadmat(file_name)
print(data.keys())

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\aqfra\\Documents\\GitRepo\\bayesee\\data\\covert-search\\large-field\\derived\\est_dp_AZ_t1.npy'

In [ ]:
target_amplitude = data["targetAmplitude"]
spot_center = data["spotCenters"]
n_location = data["nLocations"][0][0]
spot_diameter = data["spotLength"][0][0]
stimulus_design_size = data["totalLength"][0][0]
monitor_width = data["monitorPx"][0][0]
monitor_height = data["monitorPx"][0][1]
ppd = data["ppd"][0][0]
target = data["targets"]
background_mean = data["bgMean"][0][0]
background_std = data["bgContrast"][0][0] * background_mean

In [ ]:
shifted_spot_center = spot_center.copy()
shifted_spot_center[:, 0] += (monitor_height - stimulus_design_size) // 2
shifted_spot_center[:, 1] += (monitor_width - stimulus_design_size) // 2

In [ ]:
list_spot_region = [np.zeros(monitor_width, monitor_height)] * (n_location - 1)
pixel_col, pixel_row = np.meshgrid(np.arange(monitor_width), np.arange(monitor_height))

for location_index in range(n_location - 1):
    list_spot_region[location_index] = (
        pixel_row - shifted_spot_center[location_index, 0]
    ) ** 2 + (
        pixel_col - shifted_spot_center[location_index, 1]
    ) ** 2 <= spot_diameter**2 / 4

stimulus_region = np.zeros((monitor_height, monitor_width))

for location_index in range(n_location - 1):
    stimulus_region[list_spot_region[location_index]] = 1

In [ ]:
array_orientation = np.ones((n_location - 1,)) * -1

for location_index in range(1, n_location - 1):
    dy = spot_center[location_index, 0] - stimulus_design_size // 2
    dx = stimulus_design_size // 2 - spot_center[location_index, 1]
    array_orientation[location_index] = (1 - np.arctan2(dy, dx) / np.pi) / 2

orientation_hsv_color_map = np.zeros((n_location - 1, 3))
orientation_hsv_color_map[:, 0] = array_orientation

for location_index in range(n_location - 1):
    orientation_hsv_color_map[location_index, 1] = 1

    if location_index == 0:
        orientation_hsv_color_map[location_index, 2] = 0
    else:
        orientation_hsv_color_map[location_index, 2] = 0.75

rgb_orientation_color_map = hsv_to_rgb(orientation_hsv_color_map)

orientation_color_palette = dict(
    zip(array_orientation.round(3), rgb_orientation_color_map)
)

In [ ]:
prior_absent = 0.5
n_trial = 800
local_dp = subject_local_dp
n_bootstrap = 1000
n_present = int(n_trial * (1 - prior_absent))

In [ ]:
bootstrap_accuracy_all = np.zeros((n_bootstrap,))
bootstrap_hit_near = np.zeros((n_bootstrap,))
bootstrap_hit_far = np.zeros((n_bootstrap,))
bootstrap_local_hit_rate = np.zeros((n_bootstrap, n_location - 1))
bootstrap_local_miss_rate = np.zeros((n_bootstrap, n_location - 1))
bootstrap_local_fa_rate = np.zeros((n_bootstrap, n_location - 1))
bootstrap_cr_rate = np.zeros((n_bootstrap,))
bootstrap_local_present_count = np.zeros((n_bootstrap, n_location - 1))
bootstrap_local_hit_count = np.zeros((n_bootstrap, n_location - 1))
bootstrap_local_miss_count = np.zeros((n_bootstrap, n_location - 1))
bootstrap_local_fa_count = np.zeros((n_bootstrap, n_location - 1))
bootstrap_local_fh_from_count = np.zeros((n_bootstrap, n_location - 1))
bootstrap_local_fh_to_count = np.zeros((n_bootstrap, n_location - 1))

In [ ]:
for index_bootstrap in range(n_bootstrap):
    prior = np.array(
        (prior_absent, *(((1 - prior_absent) / (n_location - 1),) * (n_location - 1)))
    )
    assert np.allclose(prior.sum(), 1.0)
    log_prior_ratio = np.log(prior / prior[0])
    log_likelihood_ratio = np.zeros_like(prior)

    the_target_amplitude = target_amplitude[0, 0, 0]

    target_location = np.zeros((n_trial,), dtype=np.int64)
    target_location[:n_present] = np.random.randint(1, n_location, size=n_present)
    model_response = np.zeros_like(target_location)

    assert np.allclose(np.dot(target.flatten(), target.flatten()), 1.0)

    for index_trial in range(n_trial):
        array_standard_normal = np.random.normal(size=(n_location - 1,))
        log_likelihood_ratio[1:] = array_standard_normal * local_dp - local_dp**2 / 2
        if target_location[index_trial] > 0:
            log_likelihood_ratio[target_location[index_trial]] += (
                local_dp[target_location[index_trial] - 1] ** 2
            )

        log_posterior_ratio = log_prior_ratio + log_likelihood_ratio
        model_response[index_trial] = np.argmax(log_posterior_ratio)

    accurate_response = target_location == model_response

    array_eccentral_distance = np.zeros((n_location - 1,))

    for location_index in range(n_location - 1):
        array_eccentral_distance[location_index] = np.sqrt(
            (spot_center[location_index, 0] - stimulus_design_size // 2) ** 2
            + (spot_center[location_index, 1] - stimulus_design_size // 2) ** 2
        )

    array_eccentral_distance /= ppd

    pixel_precision_array_eccentral_distance = array_eccentral_distance.copy()

    for index_d1, distance1 in enumerate(np.unique(array_eccentral_distance)):
        for index_d2, distance2 in enumerate(
            np.unique(array_eccentral_distance)[index_d1 + 1 :]
        ):
            if distance1 != distance2 and np.abs(distance2 - distance1) < 0.5:
                pixel_precision_array_eccentral_distance[
                    array_eccentral_distance == distance2
                ] = distance1

    location_near = np.arange(1, n_location)[
        pixel_precision_array_eccentral_distance < spot_diameter * 1.5 / ppd
    ]
    location_far = np.arange(1, n_location)[
        pixel_precision_array_eccentral_distance > spot_diameter * 1.5 / ppd
    ]

    accuracy_all = accurate_response.mean()
    hit_near = accurate_response[np.isin(target_location, location_near)].mean()
    hit_far = accurate_response[np.isin(target_location, location_far)].mean()

    bootstrap_accuracy_all[index_bootstrap] = accuracy_all
    bootstrap_hit_near[index_bootstrap] = hit_near
    bootstrap_hit_far[index_bootstrap] = hit_far

    confusion_mat = confusion_matrix(
        model_response.flatten(), target_location.flatten(), labels=range(n_location)
    )

    local_hit_rate = np.array(
        [
            confusion_mat[location_index + 1, location_index + 1]
            / confusion_mat[:, location_index + 1].sum()
            for location_index in range(n_location - 1)
        ]
    )

    local_miss_rate = np.array(
        [
            confusion_mat[0, location_index + 1]
            / confusion_mat[:, location_index + 1].sum()
            for location_index in range(n_location - 1)
        ]
    )

    local_fa_rate = np.array(
        [
            confusion_mat[location_index + 1, 0] / confusion_mat[:, 0].sum()
            for location_index in range(n_location - 1)
        ]
    )

    cr = (confusion_mat[0, 0]).sum() / confusion_mat[:, 0].sum()

    bootstrap_local_hit_rate[index_bootstrap, :] = local_hit_rate
    bootstrap_local_miss_rate[index_bootstrap, :] = local_miss_rate
    bootstrap_local_fa_rate[index_bootstrap, :] = local_fa_rate
    bootstrap_cr_rate[index_bootstrap] = cr

    local_present_count = np.array(
        [
            confusion_mat[:, location_index + 1].sum()
            for location_index in range(n_location - 1)
        ]
    )

    local_hit_count = np.array(
        [
            confusion_mat[location_index + 1, location_index + 1]
            for location_index in range(n_location - 1)
        ]
    )

    local_miss_count = np.array(
        [
            confusion_mat[0, location_index + 1]
            for location_index in range(n_location - 1)
        ]
    )

    local_fa_count = np.array(
        [
            confusion_mat[location_index + 1, 0]
            for location_index in range(n_location - 1)
        ]
    )

    local_fh_from_count = np.array(
        [
            confusion_mat[1:, location_index + 1].sum()
            - confusion_mat[location_index + 1, location_index + 1]
            for location_index in range(n_location - 1)
        ]
    )

    local_fh_to_count = np.array(
        [
            confusion_mat[location_index + 1, 1:].sum()
            - confusion_mat[location_index + 1, location_index + 1]
            for location_index in range(n_location - 1)
        ]
    )

    bootstrap_local_present_count[index_bootstrap, :] = local_present_count
    bootstrap_local_hit_count[index_bootstrap, :] = local_hit_count
    bootstrap_local_miss_count[index_bootstrap, :] = local_miss_count
    bootstrap_local_fa_count[index_bootstrap, :] = local_fa_count
    bootstrap_local_fh_from_count[index_bootstrap, :] = local_fh_from_count
    bootstrap_local_fh_to_count[index_bootstrap, :] = local_fh_to_count

In [ ]:
spatial_statistics = pd.DataFrame({"id": np.tile(range(1, n_location), n_bootstrap)})
spatial_statistics["ecc"] = np.tile(
    pixel_precision_array_eccentral_distance.round(3), n_bootstrap
)

spatial_statistics["orientation"] = np.tile(array_orientation.round(3), n_bootstrap)

spatial_statistics["hit_rate"] = bootstrap_local_hit_rate.flatten()
spatial_statistics["miss_rate"] = bootstrap_local_miss_rate.flatten()
spatial_statistics["fa_rate"] = bootstrap_local_fa_rate.flatten()
spatial_statistics["present_count"] = bootstrap_local_present_count.flatten()
spatial_statistics["hit_count"] = bootstrap_local_hit_count.flatten()
spatial_statistics["miss_count"] = bootstrap_local_miss_count.flatten()
spatial_statistics["fa_count"] = bootstrap_local_fa_count.flatten()
spatial_statistics["fh_from_count"] = bootstrap_local_fh_from_count.flatten()
spatial_statistics["fh_to_count"] = bootstrap_local_fh_to_count.flatten()

In [ ]:
def quantile_interval(x, coverage=0.95):
    return np.array(
        [np.quantile(x, 0.5 - coverage / 2), np.quantile(x, 0.5 + coverage / 2)]
    )


for statistic in [
    "hit_rate",
    "miss_rate",
    "fa_rate",
    "present_count",
    "hit_count",
    "miss_count",
    "fa_count",
    "fh_from_count",
    "fh_to_count",
]:
    fig, ax = plt.subplots()
    sns.barplot(
        data=spatial_statistics,
        x="ecc",
        y=statistic,
        hue="orientation",
        palette=orientation_color_palette,
        ax=ax,
        errwidth=3,
        capsize=0.2,
        errorbar=quantile_interval,
    )
    ax.legend_.remove()
    ax.set(xlabel="Eccentral distance (deg)")

    # ax.patches[0].set_x(ax.patches[0].get_x() + 0.375)

    if statistic in ["hit_rate", "hit_count"]:
        inner_ax = fig.add_axes([0.756, 0.7, 0.144, 0.2405])
    else:
        inner_ax = fig.add_axes([0.125, 0.7, 0.144, 0.2405])

    inner_ax.imshow(stimulus_region)
    inner_ax.set(xticks=[], yticks=[])

    for location_index in range(n_location - 1):
        circle = Circle(
            (
                shifted_spot_center[location_index, 1],
                shifted_spot_center[location_index, 0],
            ),
            spot_diameter / 2,
            color=rgb_orientation_color_map[location_index],
            linewidth=1,
        )
        inner_ax.add_patch(circle)

    plt.show()

In [ ]:
from scipy import stats

for x in [
    bootstrap_accuracy_all,
    bootstrap_hit_near,
    bootstrap_hit_far,
    bootstrap_cr_rate,
]:
    print(x.mean(), quantile_interval(x.flatten()))